# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-8bb8061fc8-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    #sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString
    
    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    #sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("F1 pilots explorative search") 

Consider the following exploratory information need:

> explore F1 pilots, their nationality, racing teams, and the country of the parent organization of the team

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P17`     | country       | predicate |
| `wd:Q10497835`| F1 Team       | node |
| `wd:Q10841764`| F1 Driver       | node |
| `wd:Q9671`    | Michael Schumacher      | node |
| `wd:Q169898`  | Scuderia Ferrari         | node |
| `wd:Q27586`   |  Ferrari         | node |
| `wd:Q183`     | Germany       | node |


Also consider

```
?p wdt:P106 wd:Q10841764 .
?p wdt:P27 wd:Q183 .
```

is the BGP to retrieve all **citizens of Germany**

```
?p wdt:P17 wd:Q183 .
?p wdt:P31 wd:Q10497835 .

``` 

is the BGP to retrieve all **F1 teams of Germany**

## Workload Goals

1. Identify the BGP for obtaining the team of a given  F1 driver  and also the F1 competitions in which they participated

2. Identify the BGP that connects a F1 team to a parent company

3. How many F1 drivers and F1 teams exist in each country?

4. Compare number of F1 drivers to their nationality
 
   4.1 Is there some driver that joined a team in their own country?
   
   4.2 Which team employed the larger number of drivers?
   
   4.3 For each country , which are the teams and drivers with the highest number of F1 race participations


In [3]:
#first of all i want to figure out the difference between "scuderia ferrari" and "ferrari", i guess they will belong to different classes
q = f"""
select ?name where{{
wd:Q169898 wdt:P31 ?class .
?class <http://schema.org/name> ?name
}}
"""
run_query(q)

[('name', 'Formula One team')]
[('name', 'auto racing team')]


2

In [4]:
q = f"""
select ?name where{{
wd:Q27586 wdt:P31 ?class .
?class <http://schema.org/name> ?name
}}
"""
run_query(q)

[('name', 'racecar constructor')]
[('name', 'automobile manufacturer')]


2

In [5]:
#since this notebook is about formula 1, let's get the uri of the class "Formula One team"
q = f"""
select ?class ?name where{{
wd:Q169898 wdt:P31 ?class .
?class <http://schema.org/name> ?name
}}
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q10497835'), ('name', 'Formula One team')]
[('class', 'http://www.wikidata.org/entity/Q20074337'), ('name', 'auto racing team')]


2

In [6]:
Team = 'wd:Q10497835'
#now let's figure out if "scuderia ferrari" is connected to "Ferrari" with some property
q = f"""
select distinct ?prop ?name where{{
wd:Q169898 ?prop wd:Q27586 .
?prop <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('prop', 'http://www.wikidata.org/prop/direct/P749'), ('name', 'parent organization')]


1

In [7]:
#we now have a way to connect a team with its parent organization
parent = 'wdt:P749'
#let's list (some) formula 1 teams with the related parent company
q = f"""
select ?team ?teamName ?parent ?parentName where{{
?team wdt:P749 ?parent ;
    wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName .
?parent <http://schema.org/name> ?parentName .
}}
limit 50
"""
run_query(q)

[('team', 'http://www.wikidata.org/entity/Q172030'), ('teamName', 'McLaren'), ('parent', 'http://www.wikidata.org/entity/Q173146'), ('parentName', 'McLaren Technology Group')]
[('team', 'http://www.wikidata.org/entity/Q2704273'), ('teamName', 'Aston Martin F1 Team'), ('parent', 'http://www.wikidata.org/entity/Q27074'), ('parentName', 'Aston Martin Lagonda')]
[('team', 'http://www.wikidata.org/entity/Q169898'), ('teamName', 'Scuderia Ferrari'), ('parent', 'http://www.wikidata.org/entity/Q27586'), ('parentName', 'Ferrari')]
[('team', 'http://www.wikidata.org/entity/Q173314'), ('teamName', 'Enzo Coloni Racing Car Systems'), ('parent', 'http://www.wikidata.org/entity/Q1631454'), ('parentName', 'Scuderia Coloni')]


4

In [8]:
#something's wrong, let's list only f1 teams
q = f"""
select distinct ?team ?teamName where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName .
}}
limit 50
"""
run_query(q)

[('team', 'http://www.wikidata.org/entity/Q141818'), ('teamName', 'Scuderia Toro Rosso')]
[('team', 'http://www.wikidata.org/entity/Q173663'), ('teamName', 'Red Bull Racing')]
[('team', 'http://www.wikidata.org/entity/Q142381'), ('teamName', 'Minardi')]
[('team', 'http://www.wikidata.org/entity/Q171320'), ('teamName', 'British American Racing')]
[('team', 'http://www.wikidata.org/entity/Q171420'), ('teamName', 'HRT Formula One Team')]
[('team', 'http://www.wikidata.org/entity/Q171992'), ('teamName', 'Tyrrell Racing')]
[('team', 'http://www.wikidata.org/entity/Q172765'), ('teamName', 'Spyker F1')]
[('team', 'http://www.wikidata.org/entity/Q172974'), ('teamName', 'Forti')]
[('team', 'http://www.wikidata.org/entity/Q173012'), ('teamName', 'British Racing Motors')]
[('team', 'http://www.wikidata.org/entity/Q173054'), ('teamName', 'Midland F1 Racing')]
[('team', 'http://www.wikidata.org/entity/Q173177'), ('teamName', 'March Engineering')]
[('team', 'http://www.wikidata.org/entity/Q174008'),

50

In [9]:
#let' try and get the parent of "Lotus F1"
Lotus = 'wd:Q173018'
q = f"""
select distinct ?parent ?name where{{
wd:Q173018 wdt:P749 ?parent .
?parent <http://schema.org/name> ?name .
}}
"""
run_query(q)

Empty


0

In [10]:
#let's try to invert the relationship just for the sake of it
q = f"""
select distinct ?parent ?name where{{
?parent wdt:P749 wd:Q173018 .
?parent <http://schema.org/name> ?name .
}}
"""
run_query(q)

Empty


0

In [11]:
#let's see all the properties going out of Lotus
q = f"""
select distinct ?prop ?name where{{
wd:Q173018 ?prop ?o .
?prop <http://schema.org/name> ?name
}}
"""
run_query(q)

[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('prop', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P856'), ('name', 'official website')]
[('prop', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('prop', 'http://www.wikidata.org/prop/direct/P935'), ('name', 'Commons gallery')]


12

In [12]:
#the relationship "part of" may be promising
partOf = 'wdt:P361'
q = f"""
select distinct ?o ?name where{{
wd:Q173018 wdt:P361 ?o .
?o <http://schema.org/name> ?name
}}
"""
run_query(q)

[('o', 'http://www.wikidata.org/entity/Q25412278'), ('name', 'Lotus in Formula One')]


1

In [13]:
#what did we find?
tmp = 'wd:Q25412278'
q = f"""
select ?c ?name where{{
wd:Q25412278 wdt:P31 ?c .
?c <http://schema.org/name> ?name
}}
"""
run_query(q)

[('c', 'http://www.wikidata.org/entity/Q21484471'), ('name', 'Wikipedia article covering multiple topics')]


1

In [14]:
#maybe Lotus is object of some property connecting it to a parent company
#let's see if there is a parent company as a subject of some triple
q = f"""
select distinct ?s ?sName where{{
?s ?prop wd:Q173018 .
?s <http://schema.org/name> ?sName .
}}
"""
run_query(q)

[('s', 'http://www.wikidata.org/entity/Q3001169'), ('sName', 'Team Lotus')]
[('s', 'http://www.wikidata.org/entity/Q173996'), ('sName', 'Team Lotus')]
[('s', 'http://www.wikidata.org/entity/Q25412278'), ('sName', 'Lotus in Formula One')]
[('s', 'http://www.wikidata.org/entity/Q9213492'), ('sName', 'Category:Lotus F1 Team')]
[('s', 'http://www.wikidata.org/entity/Q29419331'), ('sName', 'Julien Simon-Chautemps')]


5

In [15]:
tmp1 = 'wd:Q3001169'
tmp2 = 'wd:Q173996'
#let's see the name of the properties connecting tmp1 and tmp2 to Lotus
q = f"""
select distinct ?prop ?name where{{
wd:Q3001169 ?prop wd:Q173018 .
?prop <http://schema.org/name> ?name
}}
"""
run_query(q)

[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]


1

In [16]:
q = f"""
select distinct ?prop ?name where{{
wd:Q173996 ?prop wd:Q173018 .
?prop <http://schema.org/name> ?name
}}
"""
run_query(q)

[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]


1

In [17]:
#it appears there is no parent company connected to Lotus, let's try and see what happens if we consider all F1 teams
q = f"""
select distinct ?prop ?name where{{
?team wdt:P31 wd:Q10497835 .
?team ?prop ?o .
?prop <http://schema.org/name> ?name
}}
"""
run_query(q)

[('prop', 'http://www.wikidata.org/prop/direct/P9279'), ('name', 'Egapro gender equality index')]
[('prop', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('prop', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1128'), ('name', 'employees')]
[('prop', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('prop', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('prop', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('prop', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P1382'), ('name', 'partially coincident with')

73

In [18]:
#here are some interesting properties that may be useful later, along with some that may allow us
#to find the parent company of a team
'''
[('prop', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('prop', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('prop', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('prop', 'http://www.wikidata.org/prop/direct/P576'), ('name', 'dissolved, abolished or demolished date')]
[('prop', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('prop', 'http://www.wikidata.org/prop/direct/P355'), ('name', 'subsidiary')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
'''
foundedBy = 'wdt:P112'
ownedBy = 'wdt:P127'
facetOf = 'wdt:P1269'
headLocation = 'wdt:P159'
dissolved = 'wdt:P576'
inception = 'wdt:P571'
subsidiary = 'wdt:P355'
country = 'wdt:P17'

#let's try and list for every team its founder
q = f"""
select ?teamName ?founder ?founderName where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName ;
    wdt:P112 ?founder .
?founder <http://schema.org/name> ?founderName .
}}
limit 50
"""
run_query(q)

[('teamName', 'British Racing Motors'), ('founder', 'http://www.wikidata.org/entity/Q1856022'), ('founderName', 'Raymond Mays')]
[('teamName', 'Williams Grand Prix Engineering'), ('founder', 'http://www.wikidata.org/entity/Q167269'), ('founderName', 'Patrick Head')]
[('teamName', 'HRT Formula One Team'), ('founder', 'http://www.wikidata.org/entity/Q171456'), ('founderName', 'Adrián Campos')]
[('teamName', 'Larrousse'), ('founder', 'http://www.wikidata.org/entity/Q171768'), ('founderName', 'Gérard Larrousse')]
[('teamName', 'Brabham'), ('founder', 'http://www.wikidata.org/entity/Q2125'), ('founderName', 'Jack Brabham')]
[('teamName', 'Williams Grand Prix Engineering'), ('founder', 'http://www.wikidata.org/entity/Q173214'), ('founderName', 'Frank Williams')]
[('teamName', 'McLaren'), ('founder', 'http://www.wikidata.org/entity/Q173081'), ('founderName', 'Bruce McLaren')]
[('teamName', 'Dallara'), ('founder', 'http://www.wikidata.org/entity/Q258628'), ('founderName', 'Gian Paolo Dallara')

18

In [19]:
#those are people, not parent companies
#let' try now with "ownedBy", i suspect it will still be people
q = f"""
select ?teamName ?owner ?ownerName where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName ;
    wdt:P127 ?owner .
?owner <http://schema.org/name> ?ownerName .
}}
limit 50
"""
run_query(q)
#it's not clear to me the difference between "parent company" and "owned by", i'm no expert
#however we're making progress

[('teamName', 'Red Bull Racing'), ('owner', 'http://www.wikidata.org/entity/Q698639'), ('ownerName', 'Red Bull GmbH')]
[('teamName', 'Alpine F1 Team'), ('owner', 'http://www.wikidata.org/entity/Q6686'), ('ownerName', 'Renault')]
[('teamName', 'Williams Grand Prix Engineering'), ('owner', 'http://www.wikidata.org/entity/Q98714466'), ('ownerName', 'Dorilton Capital')]


3

In [20]:
#let's see with facetOf
q = f"""
select ?teamName ?facet ?facetName where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName ;
    wdt:P1269 ?facet .
?facet <http://schema.org/name> ?facetName .
}}
limit 50
"""
run_query(q)

[('teamName', 'Aston Martin F1 Team'), ('facet', 'http://www.wikidata.org/entity/Q25412511'), ('facetName', 'Aston Martin in Formula One')]
[('teamName', 'Aston Martin F1 Team'), ('facet', 'http://www.wikidata.org/entity/Q25412511'), ('facetName', 'Aston Martin in Formula One')]
[('teamName', 'Renault F1 Team'), ('facet', 'http://www.wikidata.org/entity/Q65954940'), ('facetName', 'Renault in Formula One')]
[('teamName', 'Porsche'), ('facet', 'http://www.wikidata.org/entity/Q10351721'), ('facetName', 'Porsche in Formula One')]
[('teamName', 'Alfa Romeo'), ('facet', 'http://www.wikidata.org/entity/Q65960697'), ('facetName', 'Alfa Romeo in Formula One')]
[('teamName', 'Bugatti F1'), ('facet', 'http://www.wikidata.org/entity/Q104630122'), ('facetName', 'Bugatti in Formula One')]
[('teamName', 'Mercedes'), ('facet', 'http://www.wikidata.org/entity/Q65954812'), ('facetName', 'Mercedes-Benz in Formula One')]


7

In [21]:
#finally let's see "subsidiary"
q = f"""
select ?teamName ?subsidiary ?subsName where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName ;
    wdt:P355 ?subsidiary .
?subsidiary <http://schema.org/name> ?subsName .
}}
limit 50
"""
run_query(q)

[('teamName', 'Red Bull Racing'), ('subsidiary', 'http://www.wikidata.org/entity/Q2079949'), ('subsName', 'Red Bull Junior Team')]
[('teamName', 'Scuderia Ferrari'), ('subsidiary', 'http://www.wikidata.org/entity/Q5445262'), ('subsName', 'Ferrari Driver Academy')]


2

In [23]:
#let's count how many f1 teams there are and then let's see how many have a country connected to them
q = f"""
select (count(*) as ?howmany) where{{
?team wdt:P31 wd:Q10497835
}}
"""
run_query(q)

[('howmany', '67')]


1

In [24]:
q = f"""
select (count(*) as ?howmany) where{{
?team wdt:P31 wd:Q10497835 .
?team wdt:P17 ?country
}}
"""
run_query(q)

[('howmany', '60')]


1

In [25]:
#which teams don't have a country?
q = f"""
select ?team ?teamName where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName .
filter not exists 
    {{
        ?team wdt:P17 ?country
    }}
}}
"""
run_query(q)

[('team', 'http://www.wikidata.org/entity/Q61979'), ('teamName', 'Panasonic Toyota Racing')]
[('team', 'http://www.wikidata.org/entity/Q842690'), ('teamName', 'Porsche')]
[('team', 'http://www.wikidata.org/entity/Q179408'), ('teamName', 'Virgin Racing')]
[('team', 'http://www.wikidata.org/entity/Q84562599'), ('teamName', 'Alfa Romeo F1 team (2019)')]
[('team', 'http://www.wikidata.org/entity/Q4097758'), ('teamName', 'Bugatti F1')]
[('team', 'http://www.wikidata.org/entity/Q61467970'), ('teamName', 'Alfa Romeo F1 team (1950-51)')]
[('team', 'http://www.wikidata.org/entity/Q98930155'), ('teamName', 'Alpine F1 Team')]
[('team', 'http://www.wikidata.org/entity/Q61957608'), ('teamName', 'Racing Point Force India')]
[('team', 'http://www.wikidata.org/entity/Q88543639'), ('teamName', 'Onyx-Moneytron')]


9

In [26]:
#let's finish by checking headLocation
q = f"""
select ?teamName ?l ?name where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName ;
    wdt:P159 ?l .
?l <http://schema.org/name> ?name
}}
limit 50
"""
run_query(q)

[('teamName', 'Brawn GP'), ('l', 'http://www.wikidata.org/entity/Q671557'), ('name', 'Brackley')]
[('teamName', 'Williams Grand Prix Engineering'), ('l', 'http://www.wikidata.org/entity/Q991626'), ('name', 'Grove')]
[('teamName', 'Scuderia AlphaTauri Grimm'), ('l', 'http://www.wikidata.org/entity/Q52981'), ('name', 'Faenza')]
[('teamName', 'McLaren'), ('l', 'http://www.wikidata.org/entity/Q646225'), ('name', 'Woking')]
[('teamName', 'Lola Cars'), ('l', 'http://www.wikidata.org/entity/Q685640'), ('name', 'Huntingdon')]
[('teamName', 'Fondmetal'), ('l', 'http://www.wikidata.org/entity/Q1210'), ('name', 'Lombardy')]
[('teamName', 'Honda'), ('l', 'http://www.wikidata.org/entity/Q1490'), ('name', 'Tokyo')]
[('teamName', 'Leyton House Racing'), ('l', 'http://www.wikidata.org/entity/Q1490'), ('name', 'Tokyo')]
[('teamName', 'Honda'), ('l', 'http://www.wikidata.org/entity/Q727'), ('name', 'Amsterdam')]
[('teamName', 'Brabham'), ('l', 'http://www.wikidata.org/entity/Q204234'), ('name', 'Milton 

16

In [27]:
#i guess this could be another way of finding the country of origin of the team, provided
#the headquarters location is in the country of origin. let's check. 
q = f"""
select ?teamName ?countryName where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName ;
    wdt:P159 ?location .
?location wdt:P17 ?country .
?country <http://schema.org/name> ?countryName .
}}
order by asc(?teamName)
"""
run_query(q)

[('teamName', 'Brabham'), ('countryName', 'United Kingdom')]
[('teamName', 'Brawn GP'), ('countryName', 'United Kingdom')]
[('teamName', 'Dallara'), ('countryName', 'Italy')]
[('teamName', 'Fondmetal'), ('countryName', 'Italy')]
[('teamName', 'Honda'), ('countryName', 'United Kingdom')]
[('teamName', 'Honda'), ('countryName', 'Japan')]
[('teamName', 'Honda'), ('countryName', 'Kingdom of the Netherlands')]
[('teamName', 'Honda'), ('countryName', 'Netherlands')]
[('teamName', 'Leyton House Racing'), ('countryName', 'Japan')]
[('teamName', 'Ligier'), ('countryName', 'France')]
[('teamName', 'Lola Cars'), ('countryName', 'United Kingdom')]
[('teamName', 'McLaren'), ('countryName', 'United Kingdom')]
[('teamName', 'Racing Point F1 Team'), ('countryName', 'United Kingdom')]
[('teamName', 'Red Bull Racing'), ('countryName', 'United Kingdom')]
[('teamName', 'Sauber'), ('countryName', 'Switzerland')]
[('teamName', 'Scuderia AlphaTauri Grimm'), ('countryName', 'Italy')]
[('teamName', 'Williams G

17

In [29]:
#let's focus now on drivers. let's list them
q = f"""
select ?driver ?name where{{
?driver wdt:P31 wd:Q10841764 ;
    <http://schema.org/name> ?name .
}}
limit 50
"""
run_query(q)

Empty


0

In [30]:
#we know that schumacher should be a driver, let's check his classes
q = f"""
select ?class ?className where{{
wd:Q9671 wdt:P31 ?class .
?class <http://schema.org/name> ?className .
}}
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q5'), ('className', 'human')]


1

In [31]:
#maybe Driver is a profession?
q = f"""
select ?driver ?name where{{
?driver wdt:P106 wd:Q10841764 ;
    <http://schema.org/name> ?name
}}
limit 50
"""
run_query(q)

[('driver', 'http://www.wikidata.org/entity/Q49235'), ('name', 'Adrian Sutil')]
[('driver', 'http://www.wikidata.org/entity/Q172724'), ('name', 'Bernie Ecclestone')]
[('driver', 'http://www.wikidata.org/entity/Q172057'), ('name', 'Reg Parnell')]
[('driver', 'http://www.wikidata.org/entity/Q171294'), ('name', 'Vitantonio Liuzzi')]
[('driver', 'http://www.wikidata.org/entity/Q172001'), ('name', 'Tiago Monteiro')]
[('driver', 'http://www.wikidata.org/entity/Q84178'), ('name', 'Christian Klien')]
[('driver', 'http://www.wikidata.org/entity/Q172557'), ('name', 'Paul Goldsmith')]
[('driver', 'http://www.wikidata.org/entity/Q544856'), ('name', 'Carlos Sainz Jr')]
[('driver', 'http://www.wikidata.org/entity/Q3128'), ('name', 'Graham Hill')]
[('driver', 'http://www.wikidata.org/entity/Q172119'), ('name', 'André Pilette')]
[('driver', 'http://www.wikidata.org/entity/Q1866907'), ('name', 'Lloyd Casner')]
[('driver', 'http://www.wikidata.org/entity/Q3153'), ('name', 'John Surtees')]
[('driver', 'h

50

In [32]:
#i know Schumacher was a driver for Ferrari, let's find the property connecting him to the team
q = f"""
select distinct ?p ?name where{{
wd:Q9671 ?p wd:Q169898 .
?p <http://schema.org/name> ?name
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]


1

In [33]:
#let's see if the inverse gives us something more
q = f"""
select distinct ?p ?name where{{
wd:Q169898 ?p wd:Q9671 .
?p <http://schema.org/name> ?name
}}
"""
run_query(q)

Empty


0

In [34]:
member = 'wdt:P54'
#sebastian vettel was first in red bull, then in ferrari, let's see if we can find this information
q = f"""
select distinct ?driver ?driverName ?team ?teamName where{{
?driver wdt:P106 wd:Q10841764 ;
    <http://schema.org/name> ?driverName ;
    wdt:P54 ?team .
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName .
filter regex(?driverName, 'sebastian vettel', 'i')
}}
"""
run_query(q)

[('driver', 'http://www.wikidata.org/entity/Q42311'), ('driverName', 'Sebastian Vettel'), ('team', 'http://www.wikidata.org/entity/Q141818'), ('teamName', 'Scuderia Toro Rosso')]
[('driver', 'http://www.wikidata.org/entity/Q42311'), ('driverName', 'Sebastian Vettel'), ('team', 'http://www.wikidata.org/entity/Q173663'), ('teamName', 'Red Bull Racing')]
[('driver', 'http://www.wikidata.org/entity/Q42311'), ('driverName', 'Sebastian Vettel'), ('team', 'http://www.wikidata.org/entity/Q169898'), ('teamName', 'Scuderia Ferrari')]


3

In [41]:
#let's see if we can find some property for Vettel with a name suggesting he participated in some race
Vettel = 'wd:Q42311'
q = f"""
select distinct ?p ?pName where{{
wd:Q42311 ?p ?o .
?p <http://schema.org/name> ?pName .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('pName', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('pName', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pName', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/dire

68

In [43]:
#nothing immediatelly suggesting he participated in a race
#going back up i found this however when listing all the properties for a f1 team
#[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
participate = 'wdt:P1344'
#let's try with Scuderia Ferrari
q = f"""
select distinct ?o ?name where{{
wd:Q169898 wdt:P1344 ?o .
?o <http://schema.org/name> ?name .
}}
limit(50)
"""
run_query(q)

Empty


0

In [45]:
#just to be sure, let's extend to all f1 teams
q = f"""
select distinct ?o ?name where{{
?team wdt:P1344 ?o .
?team wdt:P31 wd:Q10497835 .
?o <http://schema.org/name> ?name .
}}
limit(50)
"""
run_query(q)

[('o', 'http://www.wikidata.org/entity/Q1968'), ('name', 'Formula One')]


1

In [49]:
#Schumacher for sure participated in a race in the Autodromo di Monza, let's try to use that information
q = f"""
select distinct ?o ?name ?class ?className where{{
wd:Q9671 ?p ?o .
?o <http://schema.org/name> ?name .
?o wdt:P31 ?class .
?class <http://schema.org/name> ?className .
filter regex(?name, '.*(autodromo)*.*monza.*', 'i') .
}}
"""
run_query(q)

Empty


0

In [50]:
q = f"""
select distinct ?o ?name ?class ?className where{{
wd:Q9671 ?p ?o .
?o <http://schema.org/name> ?name .
?o wdt:P31 ?class .
?class <http://schema.org/name> ?className .
filter regex(?name, '.*monza.*', 'i') .
}}
"""
run_query(q)

Empty


0

In [51]:
q = f"""
select distinct ?o ?name ?class ?className where{{
wd:Q169898 ?p ?o .
?o <http://schema.org/name> ?name .
?o wdt:P31 ?class .
?class <http://schema.org/name> ?className .
filter regex(?name, '.*monza.*', 'i') .
}}
"""
run_query(q)

Empty


0

In [53]:
#let's see all properties entering a f1 team
q = f"""
select distinct ?p ?name where{{
?s ?p ?o .
?o wdt:P31 wd:Q10497835 .
?p <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P1382'), ('name', 'partially coincident with')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P176'), ('name', 'manufacturer')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "c

27

In [54]:
'''
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('name', 'participating team')]
[('p', 'http://www.wikidata.org/prop/direct/P3764'), ('name', 'pole position')]
'''
#we have found something interesting finally
winner = 'wdt:P1346'
participate = 'wdt:P1923'
pole = 'wdt:P3764'
#let's see ferrari's participations
q = f"""
select distinct ?o ?name where{{
wd:Q169898 wdt:P1923 ?o .
?o <http://schema.org/name> ?name .
}}
limit 20
"""
run_query(q)

Empty


0

In [55]:
#wrong, participate is a property ENTERING a team
q = f"""
select distinct ?s ?name where{{
?s wdt:P1923 wd:Q169898 .
?s <http://schema.org/name> ?name .
}}
limit 20
"""
run_query(q)

[('s', 'http://www.wikidata.org/entity/Q17455748'), ('name', '2016 Formula One World Championship')]
[('s', 'http://www.wikidata.org/entity/Q20755615'), ('name', '2017 Formula One World Championship')]
[('s', 'http://www.wikidata.org/entity/Q23051084'), ('name', '2018 Formula One World Championship')]


3

In [59]:
#let's get the class of a f1 championship
Champ2016 = 'wd:Q17455748'
q = f"""
select ?c ?name where{{
wd:Q17455748 wdt:P31 ?c .
?c <http://schema.org/name> ?name
}}
"""
run_query(q)

[('c', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]


1

In [57]:
#let's see if it's subclass of something
q = f"""
select ?c ?name where{{
wd:Q17455748 wdt:P279 ?c .
?c <http://schema.org/name> ?name
}}
"""
run_query(q)

Empty


0

In [61]:
#maybe the others have a class that's more meaningful
Champ2017 = 'wd:Q20755615'
q = f"""
select ?c ?name where{{
wd:Q20755615 wdt:P31 ?c .
?c <http://schema.org/name> ?name
}}
"""
run_query(q)

[('c', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]


1

In [60]:
#properties connected to a championship
q = f"""
select distinct ?p ?name where{{
wd:Q17455748 ?p ?o .
?p <http://schema.org/name> ?name
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P2321'), ('name', 'general classification of race participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P393'), ('name', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 

19

In [62]:
'''
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
'''
nParticipants = 'wdt:P1132'
#what is property "has part"?
hasPart = 'wdt:P527'
q = f"""
select ?o ?name where{{
wd:Q17455748 wdt:P527 ?o .
?o <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('o', 'http://www.wikidata.org/entity/Q23045163'), ('name', '2016 Japanese Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q22681246'), ('name', '2016 Bahrain Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045085'), ('name', '2016 Spanish Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045170'), ('name', '2016 Hungarian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045176'), ('name', '2016 Monaco Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045058'), ('name', '2016 Russian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045154'), ('name', '2016 British Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045194'), ('name', '2016 Austrian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045137'), ('name', '2016 Belgian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045140'), ('name', '2016 Brazilian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q23045146'), ('name', '2016 European Grand Prix')]
[('o', 'http://

21

In [63]:
#we finally have the grand prix, let's see the class of a grand prix
Italian16 = 'wd:Q23045185'
q = f"""
select ?c ?name where{{
wd:Q23045185 wdt:P31 ?c .
?c <http://schema.org/name> ?name
}}
"""
run_query(q)

[('c', 'http://www.wikidata.org/entity/Q8036'), ('name', 'Italian Grand Prix')]


1

In [65]:
#not very useful, is it a subclass of something?
ItalianGrandPrix = 'wd:Q8036'
q = f"""
select ?c ?name where{{
wd:Q8036 wdt:P279 ?c .
?c <http://schema.org/name> ?name
}}
"""
run_query(q)

[('c', 'http://www.wikidata.org/entity/Q11924610'), ('name', 'Formula One Grand Prix')]


1

In [66]:
#let's get all instances of grand prix
GrandPrix = 'wd:Q11924610'
q = f"""
select ?s ?name where{{
?s wdt:P31 wd:Q11924610 .
?s <http://schema.org/name> ?name
}}
limit 50
"""
run_query(q)

[('s', 'http://www.wikidata.org/entity/Q67123905'), ('name', '2020 Bahrain Grand Prix')]
[('s', 'http://www.wikidata.org/entity/Q96088886'), ('name', '70th Anniversary Grand Prix')]
[('s', 'http://www.wikidata.org/entity/Q67123903'), ('name', '2020 Australian Grand Prix')]
[('s', 'http://www.wikidata.org/entity/Q67123906'), ('name', '2020 Vietnamese Grand Prix')]


4

In [67]:
#again not very useful. let's check the properties exiting a grand prix
q = f"""
select distinct ?p ?name where{{
wd:Q23045185 ?p ?o .
?p <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P2321'), ('name', 'general classification of race participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3157'), ('name', 'event distance')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P3764'), ('name', 'pole position')]
[('p', 'http://www.wikidata.org/prop/

14

In [68]:
classification = 'wdt:P2321'
distance = 'wdt:P3157' #is this the total distance?
#let's see the classification
q = f"""
select ?o ?name where{{
wd:Q23045185 wdt:P2321 ?o .
?o <http://schema.org/name> ?name
}}
"""
run_query(q)

[('o', 'http://www.wikidata.org/entity/Q777319'), ('name', 'Felipe Nasr')]
[('o', 'http://www.wikidata.org/entity/Q544856'), ('name', 'Carlos Sainz Jr')]
[('o', 'http://www.wikidata.org/entity/Q82816'), ('name', 'Daniel Ricciardo')]
[('o', 'http://www.wikidata.org/entity/Q76547'), ('name', 'Nico Hülkenberg')]
[('o', 'http://www.wikidata.org/entity/Q15074165'), ('name', 'Esteban Ocon')]
[('o', 'http://www.wikidata.org/entity/Q42311'), ('name', 'Sebastian Vettel')]
[('o', 'http://www.wikidata.org/entity/Q10510'), ('name', 'Jenson Button')]
[('o', 'http://www.wikidata.org/entity/Q11192'), ('name', 'Kimi Räikkönen')]
[('o', 'http://www.wikidata.org/entity/Q75820'), ('name', 'Nico Rosberg')]
[('o', 'http://www.wikidata.org/entity/Q82652'), ('name', 'Felipe Massa')]
[('o', 'http://www.wikidata.org/entity/Q9673'), ('name', 'Lewis Hamilton')]
[('o', 'http://www.wikidata.org/entity/Q10514'), ('name', 'Fernando Alonso')]
[('o', 'http://www.wikidata.org/entity/Q138745'), ('name', 'Valtteri Bottas

22

In [71]:
#for now we can answer point 1.
#let's get for each formula 1 driver the respective team
q = f"""
select ?driverName ?teamName where{{
?driver wdt:P106 wd:Q10841764 ;
    wdt:P54 ?team ;
    <http://schema.org/name> ?driverName .
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName .
}}
limit 100
"""
run_query(q)

[('driverName', 'Reg Parnell'), ('teamName', 'Alfa Romeo')]
[('driverName', 'Carlos Sainz Jr'), ('teamName', 'Scuderia Toro Rosso')]
[('driverName', 'Carlos Sainz Jr'), ('teamName', 'McLaren')]
[('driverName', 'Carlos Sainz Jr'), ('teamName', 'Renault F1 Team')]
[('driverName', 'Graham Hill'), ('teamName', 'British Racing Motors')]
[('driverName', 'Graham Hill'), ('teamName', 'Equipe Matra Sports')]
[('driverName', 'Graham Hill'), ('teamName', 'Brabham')]
[('driverName', 'John Surtees'), ('teamName', 'British Racing Motors')]
[('driverName', 'John Surtees'), ('teamName', 'Honda')]
[('driverName', 'John Surtees'), ('teamName', 'Scuderia Ferrari')]
[('driverName', 'Ayrton Senna'), ('teamName', 'Williams Grand Prix Engineering')]
[('driverName', 'Ayrton Senna'), ('teamName', 'McLaren')]
[('driverName', 'James Hunt'), ('teamName', 'McLaren')]
[('driverName', 'Riccardo Patrese'), ('teamName', 'Alfa Romeo')]
[('driverName', 'Niki Lauda'), ('teamName', 'British Racing Motors')]
[('driverName'

100

In [72]:
#we have repetitions since drivers move teams during the years
#let's try and find the property connecting a grand prix with a team
q = f"""
select distinct ?p ?name where{{
wd:Q23045185 ?p ?o .
?o wdt:P31 wd:Q10497835 .
?p <http://schema.org/name> ?name
}}
"""
run_query(q)

Empty


0

In [73]:
#let's try the inverse
q = f"""
select distinct ?p ?name where{{
?team ?p wd:Q23045185 .
?team wdt:P31 wd:Q10497835 .
?p <http://schema.org/name> ?name
}}
"""
run_query(q)

Empty


0

In [75]:
#a team does not seem to "participate" in a grand prix, does a driver participate instead?
q = f"""
select distinct ?p ?name where{{
?driver ?p wd:Q23045185 .
?driver wdt:P106 wd:Q10841764 .
?p <http://schema.org/name> ?name .
}}
"""
run_query(q)

Empty


0

In [76]:
#inverse
q = f"""
select distinct ?p ?name where{{
wd:Q23045185 ?p ?driver .
?driver wdt:P106 wd:Q10841764 .
?p <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P2321'), ('name', 'general classification of race participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P3764'), ('name', 'pole position')]
[('p', 'http://www.wikidata.org/prop/direct/P5053'), ('name', 'fastest lap')]


4

In [77]:
#we still don't have a straightforward property connecting a driver with a grand prix
#let's try again considering all drivers and using regex
q = f"""
select distinct ?p ?name where{{
?driver wdt:P106 wd:Q10841764 .
?driver ?p ?something .
?p <http://schema.org/name> ?name .
filter regex(?name, '.*(partici|race|compet|part).*')
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P3927'), ('name', 'eWRC-results.com racer ID')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('name', 'unmarried partner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P3074'), ('name', "Grace's Guide ID")]
[('p', 'http://www.wikidata.org/prop/direct/P3928'), ('name', 'MotoGP racer ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8578'), ('name', 'TracesOfWar person ID')]


9

In [78]:
#we have found property "participant in" that was not there when we used only sebastian vettel
participantIn = 'wdt:P1344'
q = f"""
select ?driverName ?object ?objectName where{{
?driver wdt:P106 wd:Q10841764 ;
    wdt:P1344 ?object .
?object <http://schema.org/name> ?objectName .
}}
limit 50
"""
run_query(q)

[('object', 'http://www.wikidata.org/entity/Q149844'), ('objectName', 'GP3 Series')]
[('object', 'http://www.wikidata.org/entity/Q149844'), ('objectName', 'GP3 Series')]
[('object', 'http://www.wikidata.org/entity/Q17455748'), ('objectName', '2016 Formula One World Championship')]
[('object', 'http://www.wikidata.org/entity/Q22833485'), ('objectName', "2015 FIA World Endurance Drivers' Championship")]
[('object', 'http://www.wikidata.org/entity/Q22833485'), ('objectName', "2015 FIA World Endurance Drivers' Championship")]
[('object', 'http://www.wikidata.org/entity/Q22833485'), ('objectName', "2015 FIA World Endurance Drivers' Championship")]
[('object', 'http://www.wikidata.org/entity/Q22833485'), ('objectName', "2015 FIA World Endurance Drivers' Championship")]
[('object', 'http://www.wikidata.org/entity/Q69343'), ('objectName', '1974 Formula One season')]
[('object', 'http://www.wikidata.org/entity/Q69641'), ('objectName', '1989 Formula One season')]
[('object', 'http://www.wikidata

50

In [80]:
#let's filter by name
participantIn = 'wdt:P1344'
q = f"""
select ?driverName ?object ?objectName where{{
?driver wdt:P106 wd:Q10841764 ;
    wdt:P1344 ?object ;
    <http://schema.org/name> ?driverName .
?object <http://schema.org/name> ?objectName .
filter regex(?objectName, '.*formula one.*', 'i')
}}
limit 50
"""
run_query(q)

[('driverName', 'Fernando Alonso'), ('object', 'http://www.wikidata.org/entity/Q17455748'), ('objectName', '2016 Formula One World Championship')]
[('driverName', 'Eppie Wietzes'), ('object', 'http://www.wikidata.org/entity/Q69343'), ('objectName', '1974 Formula One season')]
[('driverName', 'Ayrton Senna'), ('object', 'http://www.wikidata.org/entity/Q69641'), ('objectName', '1989 Formula One season')]
[('driverName', 'Luis Pérez-Sala'), ('object', 'http://www.wikidata.org/entity/Q69641'), ('objectName', '1989 Formula One season')]
[('driverName', 'Eppie Wietzes'), ('object', 'http://www.wikidata.org/entity/Q69229'), ('objectName', '1967 Formula One season')]
[('driverName', 'Ayrton Senna'), ('object', 'http://www.wikidata.org/entity/Q69633'), ('objectName', '1988 Formula One season')]
[('driverName', 'Luis Pérez-Sala'), ('object', 'http://www.wikidata.org/entity/Q69633'), ('objectName', '1988 Formula One season')]
[('driverName', 'Fernando Alonso'), ('object', 'http://www.wikidata.org

45

In [81]:
#this is getting difficult, not all pilots are guaranteed to have the "participant in" relationship
#furthermore those that have it are only a few, and they participate in only a few championships

#the only way left to get the grand prixes and championship to which a driver took part seems to be the relationship
#"general classification of race participants" exiting from a grand prix/championship

#let's try and get all championships to which Vettel took part of
#furthermore to be sure to get only formula one championships, we have to filter by name of the championship
#since a championship seems to only be instance of "sport season"
#i know for certain however that many drivers don't race only in formula 1
q = f"""
select distinct ?cName where{{
?champ wdt:P2321 wd:Q42311 .
?champ <http://schema.org/name> ?cName .
filter regex (?cName, '.*formula one.*champ.*', 'i')
}}
order by asc(?cName)
"""
run_query(q)

Empty


0

In [82]:
#let's try again to use classification more generally
q = f"""
select distinct ?cName where{{
?champ wdt:P2321 ?driver .
?champ <http://schema.org/name> ?cName .
}}
order by asc(?cName)
limit 10
"""
run_query(q)

[('cName', '1896 Paris – Roubaix')]
[('cName', '1903 Tour de France, stage 2')]
[('cName', '1904 Tour de France')]
[('cName', '1905 Tour de France')]
[('cName', '1906 Tour de France')]
[('cName', '1907 Tour de France')]
[('cName', "1909 Giro d'Italia")]
[('cName', '1911 Tour de France')]
[('cName', '1913 Paris–Bourges')]
[('cName', '1913 Tour of Flanders')]


10

In [83]:
q = f"""
select distinct ?cName ?driverName where{{
?champ wdt:P2321 ?driver .
?champ <http://schema.org/name> ?cName .
?driver <http://schema.org/name> ?driverName
}}
order by asc(?cName)
limit 20
"""
run_query(q)

[('cName', '1896 Paris – Roubaix'), ('driverName', 'Maurice Garin')]
[('cName', '1896 Paris – Roubaix'), ('driverName', 'Josef Fischer')]
[('cName', '1896 Paris – Roubaix'), ('driverName', 'Charles Meyer')]
[('cName', '1903 Tour de France, stage 2'), ('driverName', 'Maurice Garin')]
[('cName', '1903 Tour de France, stage 2'), ('driverName', 'Marcel Kerff')]
[('cName', '1903 Tour de France, stage 2'), ('driverName', 'Léon Georget')]
[('cName', '1903 Tour de France, stage 2'), ('driverName', 'Fernand Augereau')]
[('cName', '1904 Tour de France'), ('driverName', 'Gustave Drioul')]
[('cName', '1904 Tour de France'), ('driverName', 'Henri Cornet')]
[('cName', '1904 Tour de France'), ('driverName', 'Aloïs Catteau')]
[('cName', '1904 Tour de France'), ('driverName', 'Jean-Baptiste Dortignacq')]
[('cName', '1904 Tour de France'), ('driverName', 'Julien Maitron')]
[('cName', '1904 Tour de France'), ('driverName', 'Achille Colas')]
[('cName', '1904 Tour de France'), ('driverName', 'Louis Coolset

20

In [84]:
#let's try adding the constraint "driver is formula 1 pilot"
q = f"""
select distinct ?cName ?driverName where{{
?champ wdt:P2321 ?driver .
?champ <http://schema.org/name> ?cName .
?driver <http://schema.org/name> ?driverName .
?driver wdt:P106 wd:Q10841764 .
}}
order by asc(?cName)
limit 20
"""
run_query(q)

[('cName', '1950 British Grand Prix'), ('driverName', 'Reg Parnell')]
[('cName', '2005 Bahrain GP2 Series round'), ('driverName', 'Nico Rosberg')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Carlos Sainz Jr')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Kimi Räikkönen')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Nico Rosberg')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Lewis Hamilton')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Fernando Alonso')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Valtteri Bottas')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Romain Grosjean')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Max Verstappen')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Esteban Gutiérrez')]
[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Pascal Wehrlein')]
[('cName', '2016 Belgian Grand Prix'), ('d

20

In [85]:
#let's try and filter out vettel using its name, not the uri
q = f"""
select distinct ?cName ?driverName where{{
?champ wdt:P2321 ?driver .
?champ <http://schema.org/name> ?cName .
?driver <http://schema.org/name> ?driverName .
?driver wdt:P106 wd:Q10841764 .
filter regex(?driverName, 'sebastian vettel', 'i') .
}}
order by asc(?cName)
limit 20
"""
run_query(q)

[('cName', '2016 Belgian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2016 Canadian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2016 Italian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2019 Belgian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2020 Belgian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2020 British Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2020 Hungarian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2020 Italian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2020 Spanish Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2020 Styrian Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '2020 Tuscan Grand Prix'), ('driverName', 'Sebastian Vettel')]
[('cName', '70th Anniversary Grand Prix'), ('driverName', 'Sebastian Vettel')]


12

In [ ]:
#we have some results, vettel however participated in way more gps...
#earlier we didn't get results because we were filtering with "formula one championship"
#here indeed we have no championships, only gps
q = f"""
select distinct ?cName ?driverName where{{
?champ wdt:P2321 ?driver .
?champ <http://schema.org/name> ?cName .
?driver <http://schema.org/name> ?driverName .
?driver wdt:P106 wd:Q10841764 .
filter regex(?driverName, 'sebastian vettel', 'i') .
}}
order by asc(?cName)
limit 20
"""
run_query(q)

In [87]:
#what happens if we check the classification for a championship then?
q = f"""
select distinct ?object, ?name where{{
wd:Q20755615 wdt:P2321 ?object .
?object <http://schema.org/name> ?name
}}
"""
run_query(q)

Empty


0

In [88]:
#actually, after double checking, we never made sure that "classification" is a property exiting a championship
q = f"""
select distinct ?p ?name where{{
wd:Q20755615 ?p ?o .
?p <http://schema.org/name> ?name .
filter regex(?name, 'particip.*', 'i') .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('name', 'participating team')]


2

In [91]:
#only a team participates a championship
#so let's find all f1 gps to which Schumacher participated in
q = f"""
select distinct ?gpName ?cName where{{
?gp wdt:P2321 wd:Q9671 .
?gp <http://schema.org/name> ?gpName .
?gp wdt:P361 ?championship .
?championship <http://schema.org/name> ?cName .
filter regex(?cName, '.*formula one champ.*', 'i') .
}}
"""
run_query(q)

Empty


0

In [92]:
#let's simplify
q = f"""
select distinct ?gpName ?cName where{{
?gp wdt:P2321 wd:Q9671 .
?gp <http://schema.org/name> ?gpName .
?gp wdt:P361 ?championship .
?championship <http://schema.org/name> ?cName .
}}
"""
run_query(q)

Empty


0

In [94]:
#let's simplify
q = f"""
select distinct ?gpName where{{
?gp wdt:P2321 wd:Q9671 .
?gp <http://schema.org/name> ?gpName .

}}
"""
run_query(q)

Empty


0

In [95]:
#let's filter by name
q = f"""
select distinct ?gpName ?cName where{{
?gp wdt:P2321 ?driver .
?gp <http://schema.org/name> ?gpName .
?driver <http://schema.org/name> ?driverName .
filter regex(?driverName, 'michael schumacher' ,'i') .

}}
"""
run_query(q)

Empty


0

In [97]:
#remove the filter
q = f"""
select distinct ?gpName ?driverName where{{
?gp wdt:P2321 ?driver .
?gp <http://schema.org/name> ?gpName .
?driver <http://schema.org/name> ?driverName .
}}
limit 50
"""
run_query(q)

[('gpName', '2014 Omloop Het Nieuwsblad'), ('driverName', 'Egoitz García')]
[('gpName', '1977 Tour de Suisse'), ('driverName', 'Ueli Sutter')]
[('gpName', '1978 Tour de Suisse'), ('driverName', 'Ueli Sutter')]
[('gpName', '2013 Arctic Race of Norway'), ('driverName', 'Maxime Vantomme')]
[('gpName', '2013 Peace Race U23'), ('driverName', 'Kristian Haugaard')]
[('gpName', '2013 Paris–Bourges'), ('driverName', 'Rudy Barbier')]
[('gpName', '2016 Gooikse Pijl'), ('driverName', 'Rudy Barbier')]
[('gpName', '2017 Kampioenschap van Vlaanderen'), ('driverName', 'Rudy Barbier')]
[('gpName', '2016 Grand Prix de Fourmies'), ('driverName', 'Rudy Barbier')]
[('gpName', '2016 Paris-Tours'), ('driverName', 'Rudy Barbier')]
[('gpName', "2016 Tour de l'Ain, Stage 1"), ('driverName', 'Rudy Barbier')]
[('gpName', '2017 EuroEyes Cyclassics'), ('driverName', 'Rudy Barbier')]
[('gpName', '1977 Tour de Suisse'), ('driverName', 'Juan Pujol')]
[('gpName', '2013 Paris–Bourges'), ('driverName', 'Thomas Damuseau')

50

In [98]:
#let's try and connect the gp to a f1 championship
q = f"""
select distinct ?gpName ?driverName ?cName where{{
?gp wdt:P2321 ?driver .
?gp <http://schema.org/name> ?gpName .
?driver <http://schema.org/name> ?driverName .
?gp wdt:P361 ?championship .
?championship <http://schema.org/name> ?cName .
filter regex(?cName, '.*formula one.*', 'i') .
}}
limit 50
"""
run_query(q)

[('gpName', '2016 Belgian Grand Prix'), ('driverName', 'Felipe Nasr'), ('cName', '2016 Formula One World Championship')]
[('gpName', '2016 Canadian Grand Prix'), ('driverName', 'Felipe Nasr'), ('cName', '2016 Formula One World Championship')]
[('gpName', '2016 Italian Grand Prix'), ('driverName', 'Felipe Nasr'), ('cName', '2016 Formula One World Championship')]
[('gpName', '1950 British Grand Prix'), ('driverName', 'Reg Parnell'), ('cName', '1950 Formula One season')]
[('gpName', '2016 Belgian Grand Prix'), ('driverName', 'Carlos Sainz Jr'), ('cName', '2016 Formula One World Championship')]
[('gpName', '2016 Canadian Grand Prix'), ('driverName', 'Carlos Sainz Jr'), ('cName', '2016 Formula One World Championship')]
[('gpName', '2016 Italian Grand Prix'), ('driverName', 'Carlos Sainz Jr'), ('cName', '2016 Formula One World Championship')]
[('gpName', '2020 Spanish Grand Prix'), ('driverName', 'Carlos Sainz Jr'), ('cName', '2020 FIA Formula One World Championship')]
[('gpName', '2020 Brit

50

In [99]:
#let's try for one last time to get schumacher
q = f"""
select distinct ?gpName ?driverName ?cName where{{
?gp wdt:P2321 ?driver .
?gp <http://schema.org/name> ?gpName .
?driver <http://schema.org/name> ?driverName .
?gp wdt:P361 ?championship .
?championship <http://schema.org/name> ?cName .
filter regex(?cName, '.*formula one.*', 'i') .
filter regex(?driverName, '.*schumacher', 'i') .
}}
limit 50
"""
run_query(q)

Empty


0

In [106]:
#maybe for schumacher the relationship "participantIn" exists?
q = f"""
select ?gpName where{{
wd:Q9671 wdt:P1344 ?gp .
?gp <http://schema.org/name> ?gpName .
}}
"""
run_query(q)

[('gpName', '1994 Formula One season')]
[('gpName', '1995 Formula One season')]
[('gpName', '2000 Formula One season')]
[('gpName', '2001 Formula One season')]
[('gpName', '2002 Formula One season')]
[('gpName', '2003 Formula One season')]
[('gpName', '2004 Formula One season')]


7

In [107]:
#point 2 was answered very early in this notebook 
q = f"""
select distinct ?teamName ?parentName where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName .
optional
    {{
        ?team wdt:P749 ?parent .
        ?parent <http://schema.org/name> ?parentName .
    }}
optional
    {{
        ?team wdt:P127 ?parent .
        ?parent <http://schema.org/name> ?parentName .
    }}
}}
"""
run_query(q)

[('teamName', 'McLaren'), ('parentName', 'McLaren Technology Group')]
[('teamName', 'Aston Martin F1 Team'), ('parentName', 'Aston Martin Lagonda')]
[('teamName', 'Scuderia Ferrari'), ('parentName', 'Ferrari')]
[('teamName', 'Enzo Coloni Racing Car Systems'), ('parentName', 'Scuderia Coloni')]
[('teamName', 'Red Bull Racing'), ('parentName', 'Red Bull GmbH')]
[('teamName', 'Alpine F1 Team'), ('parentName', 'Renault')]
[('teamName', 'Williams Grand Prix Engineering'), ('parentName', 'Dorilton Capital')]
[('teamName', 'Scuderia Toro Rosso')]
[('teamName', 'Minardi')]
[('teamName', 'British American Racing')]
[('teamName', 'HRT Formula One Team')]
[('teamName', 'Tyrrell Racing')]
[('teamName', 'Spyker F1')]
[('teamName', 'Forti')]
[('teamName', 'British Racing Motors')]
[('teamName', 'Midland F1 Racing')]
[('teamName', 'March Engineering')]
[('teamName', 'Caterham F1')]
[('teamName', 'Ligier')]
[('teamName', 'Alfa Romeo')]
[('teamName', 'Haas F1 Team')]
[('teamName', 'Honda')]
[('teamName

67

In [108]:
#point 3.
#let's count for each country the number of f1 drivers

#let's try and figure out first which drivers are alive
q = f"""
select distinct ?p ?name ?oName where{{
?s wdt:P106 wd:Q10841764 .
?s ?p ?o .
?p <http://schema.org/name> ?name .
?o <http://schema.org/name> ?oName .
filter regex(?name, '.*(death|die).*', 'i') .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death'), ('oName', 'Ostend')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death'), ('oName', 'Grainau')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death'), ('oName', 'Zürich')]
[('p', 'http://www.wikidata.org/prop/direct/P509'), ('name', 'cause of death'), ('oName', 'aviation accident')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death'), ('oName', 'Santa Fe')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death'), ('oName', 'Scanzorosciate')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death'), ('oName', 'Autodromo Enzo e Dino Ferrari')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death'), ('oName', 'Torrance')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death'), ('oName', 'Saint Barthélemy')]
[('p', 'http://www.wikidata.org/prop/direc

355

In [109]:
#let's try and see if there is a more general property for alive or dead
q = f"""
select distinct ?p ?name where{{
?s wdt:P106 wd:Q10841764 .
?s ?p ?o .
?p <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P3927'), ('name', 'eWRC-results.com racer ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9129'), ('name', 'Data Collection of the Hungarian Peerage ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6682'), ('name', 'Motorsports Hall of Fame of America ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8161'), ('name', 'CONI honoured ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop

308

In [110]:
#why couldn't i find "date of death" with this regex?
q = f"""
select distinct ?p ?name where{{
?s wdt:P106 wd:Q10841764 .
?s ?p ?o .
?p <http://schema.org/name> ?name .
filter regex(?name, '.*(death).*', 'i') .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P509'), ('name', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P570'), ('name', 'date of death')]


4

In [111]:
#why couldn't i find "date of death" with this regex?
q = f"""
select distinct ?p ?name where{{
?s wdt:P106 wd:Q10841764 .
?s ?p ?o .
?p <http://schema.org/name> ?name .
filter regex(?name, '.*(death|die).*', 'i') .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P349'), ('name', 'National Diet Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5019'), ('name', 'Brockhaus Enzyklopädie online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P509'), ('name', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P570'), ('name', 'date of death')]


6

In [112]:
#let's select all alive pilots for each country
dateDeath = 'wdt:P570'
q = f"""
select ?countryName (count(*) as ?howMany) where{{
?driver wdt:P106 wd:Q10841764 .
?driver wdt:P17 ?country .
?country <http://schema.org/name> ?countryName .
filter not exists
    {{
        ?driver wdt:P570 ?date .
    }}
}}
group by ?countryName
order by desc(?howMany)
"""
run_query(q)

Empty


0

In [113]:
#let's remove the filter
dateDeath = 'wdt:P570'
q = f"""
select ?countryName (count(*) as ?howMany) where{{
?driver wdt:P106 wd:Q10841764 .
?driver wdt:P17 ?country .
?country <http://schema.org/name> ?countryName .
}}
group by ?countryName
order by desc(?howMany)
"""
run_query(q)

Empty


0

In [114]:
#a person has another property to state the country of citizenship
#from vettel we have that
citizenship = 'wdt:P27'
q = f"""
select ?countryName (count(*) as ?howMany) where{{
?driver wdt:P106 wd:Q10841764 .
?driver wdt:P27 ?country .
?country <http://schema.org/name> ?countryName .
filter not exists
    {{
        ?driver wdt:P570 ?date .
    }}
}}
group by ?countryName
order by desc(?howMany)
"""
run_query(q)

[('countryName', 'United Kingdom'), ('howMany', '62')]
[('countryName', 'Germany'), ('howMany', '26')]
[('countryName', 'United States of America'), ('howMany', '25')]
[('countryName', 'Belgium'), ('howMany', '14')]
[('countryName', 'South Africa'), ('howMany', '11')]
[('countryName', 'Kingdom of the Netherlands'), ('howMany', '11')]
[('countryName', 'Spain'), ('howMany', '9')]
[('countryName', 'Italy'), ('howMany', '9')]
[('countryName', 'Austria'), ('howMany', '9')]
[('countryName', 'Finland'), ('howMany', '8')]
[('countryName', 'Canada'), ('howMany', '7')]
[('countryName', 'France'), ('howMany', '6')]
[('countryName', 'Argentina'), ('howMany', '6')]
[('countryName', 'Ireland'), ('howMany', '5')]
[('countryName', 'Brazil'), ('howMany', '3')]
[('countryName', 'Portugal'), ('howMany', '3')]
[('countryName', 'Poland'), ('howMany', '2')]
[('countryName', 'Australia'), ('howMany', '2')]
[('countryName', 'Mexico'), ('howMany', '2')]
[('countryName', 'Thailand'), ('howMany', '2')]
[('countr

27

In [115]:
#now let's check how many f1 teams there are in each country
q = f"""
select ?countryName (count(*) as ?howMany) where{{
?team wdt:P31 wd:Q10497835 .
?team wdt:P17 ?country .
?country <http://schema.org/name> ?countryName .
filter not exists
    {{
        ?team wdt:P576 ?d .
    }}
}}
group by ?countryName
order by desc(?howMany)
"""
run_query(q)

[('countryName', 'United Kingdom'), ('howMany', '23')]
[('countryName', 'Italy'), ('howMany', '15')]
[('countryName', 'France'), ('howMany', '6')]
[('countryName', 'Netherlands'), ('howMany', '2')]
[('countryName', 'Germany'), ('howMany', '2')]
[('countryName', 'Russia'), ('howMany', '2')]
[('countryName', 'Japan'), ('howMany', '2')]
[('countryName', 'Australia'), ('howMany', '1')]
[('countryName', 'Spain'), ('howMany', '1')]
[('countryName', 'Malaysia'), ('howMany', '1')]
[('countryName', 'Switzerland'), ('howMany', '1')]


11

In [116]:
#let's check instead without filtering out the dissolved teams
q = f"""
select ?countryName (count(*) as ?howMany) where{{
?team wdt:P31 wd:Q10497835 .
?team wdt:P17 ?country .
?country <http://schema.org/name> ?countryName .
}}
group by ?countryName
order by desc(?howMany)
"""
run_query(q)

[('countryName', 'United Kingdom'), ('howMany', '26')]
[('countryName', 'Italy'), ('howMany', '16')]
[('countryName', 'France'), ('howMany', '6')]
[('countryName', 'Netherlands'), ('howMany', '2')]
[('countryName', 'Germany'), ('howMany', '2')]
[('countryName', 'Russia'), ('howMany', '2')]
[('countryName', 'Japan'), ('howMany', '2')]
[('countryName', 'Australia'), ('howMany', '1')]
[('countryName', 'Spain'), ('howMany', '1')]
[('countryName', 'Malaysia'), ('howMany', '1')]
[('countryName', 'Switzerland'), ('howMany', '1')]


11

In [117]:
#let's see which teams dissolved
q = f"""
select ?teamName ?date where{{
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName ;
    wdt:P576 ?date .
}}
"""
run_query(q)

[('teamName', 'Lola Cars'), ('date', '2012-01-01T00:00:00Z')]
[('teamName', 'Enzo Coloni Racing Car Systems'), ('date', '1991-01-01T00:00:00Z')]
[('teamName', 'Simtek'), ('date', '1995-01-01T00:00:00Z')]
[('teamName', 'Brawn GP'), ('date', '2009-01-01T00:00:00Z')]
[('teamName', 'Alfa Romeo F1 team (1950-51)'), ('date', '1951-01-01T00:00:00Z')]


5

In [119]:
#4.1 drivers that joined teams from their own original country
q = f"""
select distinct ?driverName ?teamName ?countryName where{{
?driver wdt:P106 wd:Q10841764 ;
    <http://schema.org/name> ?driverName ;
    wdt:P27 ?country1 ;
    wdt:P54 ?team .
    
?team wdt:P17 ?country2 ;
    <http://schema.org/name> ?teamName .
    
?country1 <http://schema.org/name> ?countryName .

filter(?country1 = ?country2) .
}}
"""
run_query(q)

[('driverName', 'Will Stevens'), ('teamName', 'Manor Motorsport'), ('countryName', 'United Kingdom')]
[('driverName', 'David Coulthard'), ('teamName', 'Red Bull Racing'), ('countryName', 'United Kingdom')]
[('driverName', 'Alexander Albon'), ('teamName', 'Red Bull Racing'), ('countryName', 'United Kingdom')]
[('driverName', 'Jackie Stewart'), ('teamName', 'Tyrrell Racing'), ('countryName', 'United Kingdom')]
[('driverName', 'Ivor Bueb'), ('teamName', 'British Racing Partnership'), ('countryName', 'United Kingdom')]
[('driverName', 'Innes Ireland'), ('teamName', 'British Racing Partnership'), ('countryName', 'United Kingdom')]
[('driverName', 'Trevor Taylor'), ('teamName', 'British Racing Partnership'), ('countryName', 'United Kingdom')]
[('driverName', 'Henry Taylor'), ('teamName', 'British Racing Partnership'), ('countryName', 'United Kingdom')]
[('driverName', 'Tom Bridger'), ('teamName', 'British Racing Partnership'), ('countryName', 'United Kingdom')]
[('driverName', 'Cliff Allison

121

In [120]:
#4.2 which team employed the largest amount of drivers?
q = f"""
select ?teamName (count(*) as ?n) where{{
?driver wdt:P106 wd:Q10841764 .
?driver wdt:P54 ?team .
?team wdt:P31 wd:Q10497835 ;
    <http://schema.org/name> ?teamName .
}}
group by ?teamName
order by desc(?n)
limit 10
"""
run_query(q)

[('teamName', 'Scuderia Ferrari'), ('n', '15')]
[('teamName', 'British Racing Motors'), ('n', '14')]
[('teamName', 'McLaren'), ('n', '14')]
[('teamName', 'Williams Grand Prix Engineering'), ('n', '13')]
[('teamName', 'Alfa Romeo'), ('n', '10')]
[('teamName', 'Brabham'), ('n', '9')]
[('teamName', 'Scuderia Toro Rosso'), ('n', '5')]
[('teamName', 'Red Bull Racing'), ('n', '5')]
[('teamName', 'Tyrrell Racing'), ('n', '5')]
[('teamName', 'March Engineering'), ('n', '4')]


10

In [121]:
#let's double check if scuderia ferrari employed 15 drivers in total
q = f"""
select ?driverName where{{
?driver wdt:P106 wd:Q10841764 ;
    wdt:P54 wd:Q169898 ;
    <http://schema.org/name> ?driverName .
}}
"""
run_query(q)

[('driverName', 'John Surtees')]
[('driverName', 'Niki Lauda')]
[('driverName', 'Gilles Villeneuve')]
[('driverName', 'Tony Brooks')]
[('driverName', 'Sebastian Vettel')]
[('driverName', 'Michael Schumacher')]
[('driverName', 'Kimi Räikkönen')]
[('driverName', 'Gerhard Berger')]
[('driverName', 'Fernando Alonso')]
[('driverName', 'Jacky Ickx')]
[('driverName', 'Charles Leclerc')]
[('driverName', 'Olivier Gendebien')]
[('driverName', 'Arturo Merzario')]
[('driverName', 'Mike Parkes')]
[('driverName', 'Cliff Allison')]


15

In [126]:
#4.3 for each country, the drivers with highest number of f1 race participations

#let's try again to find a property connecting a gp with drivers
q = f"""
select distinct ?property ?name where{{
?gp wdt:P361 ?championship .
?championship <http://schema.org/name> ?cName .
filter regex(?cName, '.*formula one.*', 'i') .
?gp ?property ?driver .
?driver wdt:P106 wd:Q10841764 .
?property <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('property', 'http://www.wikidata.org/prop/direct/P3342'), ('name', 'significant person')]
[('property', 'http://www.wikidata.org/prop/direct/P2321'), ('name', 'general classification of race participants')]
[('property', 'http://www.wikidata.org/prop/direct/P2417'), ('name', 'stage classification')]
[('property', 'http://www.wikidata.org/prop/direct/P3764'), ('name', 'pole position')]
[('property', 'http://www.wikidata.org/prop/direct/P710'), ('name', 'participant')]
[('property', 'http://www.wikidata.org/prop/direct/P5053'), ('name', 'fastest lap')]


7

In [136]:
#after all these trials, i missed "participant", let's see what it is
participant = 'wdt:P710'
q = f"""
select distinct ?driverName ?gpName where{{
?gp wdt:P361 ?championship .
?championship <http://schema.org/name> ?cName .
filter regex(?cName, '.*formula one.*', 'i') .
?gp wdt:P710 ?driver .
?gp <http://schema.org/name> ?gpName .
?driver wdt:P106 wd:Q10841764 .
?driver <http://schema.org/name> ?driverName
}}
limit 50
"""
run_query(q)

[('driverName', 'Carlos Sainz Jr'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Sebastian Vettel'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Kimi Räikkönen'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Lewis Hamilton'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Valtteri Bottas'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Romain Grosjean'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Alexander Albon'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Pierre Gasly'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Lance Stroll'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'George Russell'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Charles Leclerc'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Lando Norris'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Max Verstappen'), ('gpName', '2020 Austrian Grand Prix')]
[('driverName', 'Antonio

50

In [137]:
#we missed an important property

#going back up i found an interestin property exiting the 2016 championship
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
league = 'wdt:P3450'
#let's check it out
q = f"""
select ?o ?name where{{
wd:Q17455748 wdt:P3450 ?o .
?o <http://schema.org/name> ?name .
}}
"""
run_query(q)

[('o', 'http://www.wikidata.org/entity/Q1968'), ('name', 'Formula One')]


1

In [140]:
#we can try to use that property and that entity to filter out formula 1 championship instead of using regex on the name
League = 'wd:Q1968'
#let's try now and list all participations for schumacher
q = f"""
select distinct ?gpName where{{
?gp wdt:P710 wd:Q9671 .
?gp wdt:P3450 wd:Q1968 ;
    <http://schema.org/name> ?gpName .
}}
"""
run_query(q)

Empty


0

In [143]:
#let's count the participations for each driver then
q = f"""
select ?driverName (count(*) as ?n) where{{
?gp wdt:P710 ?driver .
?driver wdt:P106 wd:Q10841764 .
?driver <http://schema.org/name> ?driverName .
}}
group by ?driverName
"""
run_query(q)

[('driverName', 'Ayrton Senna'), ('n', '1')]
[('driverName', 'Jean-Louis Schlesser'), ('n', '1')]
[('driverName', 'Alexander Wurz'), ('n', '5')]
[('driverName', 'Oscar Alfredo Gálvez'), ('n', '1')]
[('driverName', 'Pedro Lamy'), ('n', '1')]
[('driverName', 'Alexander Albon'), ('n', '3')]
[('driverName', 'Lance Stroll'), ('n', '7')]
[('driverName', 'Giedo van der Garde'), ('n', '1')]
[('driverName', 'Carlos Sainz Jr'), ('n', '8')]
[('driverName', 'Sebastian Vettel'), ('n', '9')]
[('driverName', 'Nico Rosberg'), ('n', '1')]
[('driverName', 'Valtteri Bottas'), ('n', '8')]
[('driverName', 'James Hunt'), ('n', '1')]
[('driverName', 'Mark Webber'), ('n', '4')]
[('driverName', 'Marc Gené'), ('n', '3')]
[('driverName', 'Pierre Gasly'), ('n', '8')]
[('driverName', 'Sam Bird'), ('n', '2')]
[('driverName', 'John Watson'), ('n', '1')]
[('driverName', 'Kimi Räikkönen'), ('n', '8')]
[('driverName', 'Allan McNish'), ('n', '2')]
[('driverName', 'André Lotterer'), ('n', '7')]
[('driverName', 'Antonio G

40

In [144]:
#really not many participations... let's answer to point 4.3 nonetheless
q1 = f"""
select ?driver ?country (count(*) as ?n) where{{
?gp wdt:P710 ?driver .
?driver wdt:P106 wd:Q10841764 .
?driver wdt:P27 ?country .
}}
group by ?driver ?country
"""

q2 = f"""
select ?country (max(?n) as ?max) where{{
    {{
        select ?driver ?country (count(*) as ?n) where{{

?gp wdt:P710 ?driver .

?driver wdt:P106 wd:Q10841764 .

?driver wdt:P27 ?country .

}}

group by ?driver ?country



    }}
}}
"""

q3 = f"""
select ?d ?c (count(*) as ?cnt) where{{
?gp wdt:P710 ?d .
?d wdt:P106 wd:Q10841764 .
?d wdt:P27 ?c .
}}
group by ?d ?c
""" 

q4 = f"""
select ?driverName ?countryName ?max where{{
?d <http://schema.org/name> ?driverName .
?country <http://schema.org/name> ?countryName .
    {{
        select ?country (max(?n) as ?max) where{{

    {{

        select ?driver ?country (count(*) as ?n) where{{

?gp wdt:P710 ?driver .

?driver wdt:P106 wd:Q10841764 .

?driver wdt:P27 ?country .

}}

group by ?driver ?country




    }}

}}



    }}
    
    {{
        select ?d ?c (count(*) as ?cnt) where{{

?gp wdt:P710 ?d .

?d wdt:P106 wd:Q10841764 .

?d wdt:P27 ?c .

}}

group by ?d ?c



    }}
    filter(?max = ?cnt && ?country = ?c)
}}
"""

run_query(q4)

[('driverName', 'Carlos Sainz Jr'), ('countryName', 'Spain'), ('max', '8')]
[('driverName', 'Ayrton Senna'), ('countryName', 'Brazil'), ('max', '1')]
[('driverName', 'Carroll Shelby'), ('countryName', 'United States of America'), ('max', '1')]
[('driverName', 'Sebastian Vettel'), ('countryName', 'Germany'), ('max', '9')]
[('driverName', 'Kimi Räikkönen'), ('countryName', 'Finland'), ('max', '8')]
[('driverName', 'Juan Manuel Fangio'), ('countryName', 'Argentina'), ('max', '1')]
[('driverName', 'Alberto Ascari'), ('countryName', 'Kingdom of Italy'), ('max', '1')]
[('driverName', 'Mark Webber'), ('countryName', 'Australia'), ('max', '4')]
[('driverName', 'Lewis Hamilton'), ('countryName', 'United Kingdom'), ('max', '12')]
[('driverName', 'Alexander Wurz'), ('countryName', 'Austria'), ('max', '5')]
[('driverName', 'Valtteri Bottas'), ('countryName', 'Finland'), ('max', '8')]
[('driverName', 'Romain Grosjean'), ('countryName', 'Switzerland'), ('max', '2')]
[('driverName', 'Oscar Alfredo Gá

23